In [ ]:
import re
import json
import requests
from bs4 import BeautifulSoup

MAIN = 'https://app.erasmusplusols.eu/'
LOGIN_POST = 'https://app.erasmusplusols.eu/auth/login/user'
login_details = {'_token'   : '',
                 'login'    : '',
                 'password' : ''}
credentials = {'x-altissia-token'  : '',
               'x-device-uuid'     : ''}

def recieveCredentials(input):
    matches = re.findall(r"\'[^\']*\'", input)
    return [match.strip('\'') for match in matches]    
        
with requests.Session() as s:
    login_page = s.get(MAIN)
    login_soup = BeautifulSoup(login_page.text, 'html5lib')
    token = login_soup.find('input', {'name':'_token'}).get('value')
    login_details['_token'] = token
    
    dashboard_page = s.post(LOGIN_POST, data=login_details)
    dashboard_soup = BeautifulSoup(dashboard_page.text, 'html5lib')
    credential_script = dashboard_soup.find_all('script')
    credential_lines = list(credential_script[3].children)[0].get_text().splitlines()
    credentials['x-device-uuid'] = recieveCredentials(credential_lines[5])[1]
    credentials['x-altissia-token'] = recieveCredentials(credential_lines[6])[1]



In [ ]:
missions_page = s.get('https://app.erasmusplusols.eu/gw//lcapi/main/api/lc/user-learning-paths/language/en_GB', headers = credentials)
missions_dict = json.loads(missions_page.text)
lessons_list = [lessons_val['externalId']
                for mission_val in missions_dict['missions'][104:]
                for lessons_val in mission_val['lessons']]
print(json.dumps(lessons_list, sort_keys=False, indent=4))

In [ ]:
curriculum = []

for lesson in lessons_list:
    
    activities_url = f'https://app.erasmusplusols.eu/gw//lcapi/main/api/lc/lessons/{lesson}'
    
    activities_page = s.get(activities_url, headers = credentials)
    activities_dict_raw = json.loads(activities_page.text)
    activities_list = []    
    no_summary_test = False if activities_dict_raw['activities'][-1]['activityType'] == 'SUMMARY_TEST' else True
    
    for activity in activities_dict_raw['activities']:
        activity_id = activity['externalId']
        activity_dict = {'activityType' : activity['activityType'],
                         'externalId'   : activity_id,
                         'obligatoriness'   : no_summary_test}    
            
        if activity['activityType'] == 'SUMMARY_TEST' or activity['activityType'] == 'EXERCISE':
            
            answers_url = f'{activities_url}/activities/{activity_id}?translationLg=en_GB'
                
            answers_page = s.get(answers_url, headers = credentials)
            answers_dict_raw = json.loads(answers_page.text)
            answers_items_dict = answers_dict_raw['content']['items']
            answers = [item['correctAnswers'][0][0] for item in answers_items_dict]
            
            if activity['activityType'] == 'SUMMARY_TEST': 
                activity_dict['obligatoriness'] = True 
            activity_dict['answers'] = answers
            
            
        activities_list.append(activity_dict)
    
    curriculum.append({'title'      : lesson,
                       'activities' : activities_list})
    
#print(json.dumps(curriculum, sort_keys=False, indent=4))

In [ ]:
print(json.dumps(curriculum[0]['activities'], sort_keys=False, indent=4))

In [ ]:
separator = " </br>"

necessity = []

for lesson in curriculum:
    necessary_activities = [{#'Lesson'  : activity['externalId'],
                             'Type'    : activity['activityType'] + ': ' + activity['externalId'].lower(),
                             'What do' : activity.get('answers', ['Just click it'])}
                            for activity in lesson['activities'] if activity['obligatoriness'] == True]
    
    necessary_lesson = {'Title'    : lesson['title'],
                         'Must do' : necessary_activities}
    necessity.append(necessary_lesson)
    


with open('necessity.html','w') as f:
    f.write(separator.join(json.dumps(necessity, sort_keys=False, indent=4).splitlines()))

In [ ]:
def checkType(obj):
    return isinstance(obj, list) and all(isinstance(elem, str) for elem in obj)

In [ ]:
def printItems(dictObj, parent, indent, list_type = 'ul'):
    html_list = ''
    tabulation = '  ' * indent
    if len(dictObj):
        if isinstance(dictObj, str):
            #print(f'{tabulation}<li>{dictObj}</li>')
            html_list += f'{tabulation}<li>{dictObj}</li>\n'
        else:
            #print(f'{tabulation}<ul>')
            html_list += f'{tabulation}<{list_type}>\n'
            dictObj = {num : el for num, el in enumerate(dictObj)} if isinstance(dictObj, list) else dictObj
            for k,v in dictObj.items():
                html_list += printItems(v, k, indent+1, 'ol') if checkType(v) and len(v) > 1 else printItems(v, k, indent+1)
            #print(f'{tabulation}</ul>')
            html_list += f'{tabulation}</{list_type}>\n'
            
    return html_list

In [ ]:
with open('list.html','w', encoding = 'utf-8') as f:
    f.write(''.join(printItems(necessity, 'root', 0).splitlines()))

In [ ]:
with open('curriculum.json', 'w') as f:
    json.dump(curriculum, f)

In [ ]:
separator = " </br>"
with open('test.html','w', encoding = 'utf-8') as f:
    f.write(separator.join(answers))